In [85]:
from openai import OpenAI
from typing import List, Dict

def ask(messages: List[Dict], is_json: bool = False):
        client = OpenAI(api_key='sk-a6bb624707354861abff16885ffe5252', base_url="https://dashscope.aliyuncs.com/compatible-mode/v1")
        
        if is_json:
                response = client.chat.completions.create(
                        model = 'qwen-plus',
                        temperature = 0,
                        messages = messages,
                        response_format={ "type": "json_object" }
                )
        else:
                response = client.chat.completions.create(
                        model = 'qwen-plus',
                        temperature = 0,
                        messages = messages,
                )

        return response

<h2>Web Search Tool<h2

In [46]:
from search import internet_search, process_content #web search 

In [47]:
search_T = internet_search('population of Toronto in 2022')
search_T

[{'title': 'Demographics of Toronto - Wikipedia',
  'href': 'https://en.wikipedia.org/wiki/Demographics_of_Toronto',
  'body': "The last complete census by Statistics Canada, which was taken in 2021, estimated there were 2,794,356 living in Toronto, [ 25 ] making it the most populous city in Canada [ 26 ] and the fourth most populous municipality in North America. [ 27 ] Toronto's population grew by 2.3 percent from 2016 to 2021, with an annual growth rate of 0.46 percent."},
 {'title': 'Toronto - Statistics & Facts | Statista',
  'href': 'https://www.statista.com/topics/9849/toronto/',
  'body': "Premium Statistic Average age of Toronto's population in Canada 2001-2022 Premium Statistic Number of births in Toronto, Canada 2001-2022, by gender"},
 {'title': "Toronto's population is the fastest growing in Canada and U.S.: report ...",
  'href': 'https://nationalpost.com/news/canada/toronto-population-fastest-growing-canada-united-states',
  'body': 'A new report published by Toronto Met

In [48]:
search_T[0]

{'title': 'Demographics of Toronto - Wikipedia',
 'href': 'https://en.wikipedia.org/wiki/Demographics_of_Toronto',
 'body': "The last complete census by Statistics Canada, which was taken in 2021, estimated there were 2,794,356 living in Toronto, [ 25 ] making it the most populous city in Canada [ 26 ] and the fourth most populous municipality in North America. [ 27 ] Toronto's population grew by 2.3 percent from 2016 to 2021, with an annual growth rate of 0.46 percent."}

In [49]:
search_N = internet_search('population of New York City in 2022')
search_N[0]

{'title': 'New York, New York Population History | 1840 - 2022 - Biggest US Cities',
 'href': 'https://www.biggestuscities.com/city/new-york-new-york',
 'body': 'Access the most recent census population information for New York, New York, including a population profile and history. ... 2022: 8,335,798: 1-1.6% 2021: 8,467,513: 1-3.5% ... the current population of New York is 8,258,035. New York, New York is the largest city in the US. What county is New York, New York in? Parts of New York are located ...'}

In [50]:
class WebSearch:
    def __init__(self, name:str='web_search', threhold:int=8000):
        self.system_prompt = """
You are a Insight Researcher.

1. To find detail informtion for the user query
and summary the content into one sentence as simple as possible
2. If the user's question is about specific numerical values, 
only return the numerical results without any additional explanation.
"""
        self.name = name
        self.description = "the tool use for web search"
        self.threhold = threhold

    def __call__(self, query:str):
        # print(query)
        results = internet_search(query)
        # print(results)
        all_text = ""
        windows_size = 0
        for item in results:
            if windows_size >= self.threhold:
                break
            page_content = process_content(item['href'])
            for page in page_content:
                if windows_size + len(page) > self.threhold:
                    remaining_space = self.threhold - windows_size
                    all_text += page[:remaining_space].strip() + "\n\n"
                    windows_size = self.threhold
                    break
                else:
                    windows_size += len(page)
                    all_text += page + "\n\n"
            if windows_size >= self.threhold:
                break

        msg = [{"role":"system","content":self.system_prompt},
               {"role":"user", "content": f"The search results are {all_text}"}]
        
        answer = ask(messages=msg)

        return answer.choices[0].message.content

In [61]:
search_tool = WebSearch()
result1 = search_tool("population of Toronto in 2022")
print(result1)

Toronto's population was 2,794,356 in 2021, making it the most populous city in Canada.


In [62]:
result2 = search_tool("population of New York City in 2022")
print(result2)

New York, New York's current population is 8,258,035.


<h2>Math Tool<h2

<h3>llm+python</h3

In [91]:
system_prompt = """
Extract the numerical values from the strings of the minuend and subtrahend provided by the user.
"""

user_content = f"""
input parameters
<minuend>
{result1}
</minuend>

<subtrahend>
{result2}
</subtrahend>

output parameters in json format
```json
{{
    "minuend":"the value of minuend in float",
    "subtrahend": "the value of subtrahend in float"
}}
```
only return the numerical results without any additional explanation.
"""
print(user_content)
# 提取减数和被减数


input parameters
<minuend>
Toronto‘s population was 2,794,356 in 2021, making it the most populous city in Canada.
</minuend>

<subtrahend>
New York, New York’s current population is 8,258,035.
</subtrahend>

output parameters in json format
```json
{
    "minuend":"the value of minuend in float",
    "subtrahend": "the value of subtrahend in float"
}
```
only return the numerical results without any additional explanation.



In [92]:
messages = [
    {"role":"system", "content":system_prompt},
    {"role":"user", "content":user_content}
]

response = ask(messages=messages, is_json=True)
response.choices[0].message.content

'{\n    "minuend": 2794356.0,\n    "subtrahend": 8258035.0\n}'

In [90]:
type(response.choices[0].message.content)

str

In [93]:
import json
result_dict = json.loads(response.choices[0].message.content)
print(result_dict)

{'minuend': 2794356.0, 'subtrahend': 8258035.0}


In [94]:
type(result_dict)#把json字符串转换成字典

dict

In [95]:
def abs_diff(minuend:float, subtrahend:float) -> float:
    if minuend >= subtrahend:
        return minuend - subtrahend
    else:
        return subtrahend - minuend
    # 计算绝对差值
    
diff = abs_diff(result_dict['minuend'], result_dict['subtrahend'])
print(diff)

5463679.0


<h3>llm method<h3

In [107]:
system_prompt = "You are a calculation assistant."

question = f"To calculate the abs difference between {result1} and {result2} "
user_content = f"""
Answer the Question based on the Context. When you write down a expression, it MUST ONLY consists of numbers and operators. Here are some guidelines that you will be PANALIZED if you don't follow:

  - When you are asked for differences, you consider the absolute value of the difference. Difference of two numbers is always positive.For instance, the difference between 1 and 2 is 1, not -1.
  - When you are applying operations (e.g. difference, summation, ratio, etc.) between multiple values in the Context, you must unify the units of those numbers. For instance, you cannot add 1 meter to 1 foot.
     - You must pick the values in the same units if all the values are available in the same units.
     - If not, you must convert them to the same units before applying the operation.
  - You MUST strictly follow the unit (e.g. meter, kilometer, million, etc.) you were asked.
     - If the Context has the numbers in same units as the question, you can directly use them.
     - If the Context has the numbers in different units than the question, you must convert them to the units asked in the question.For example, if the question asks for the distance between two cities in kilometers, but the Context has the distance in miles, you must convert the distance to kilometers.
  - If you are asked about a particular number in millions, billions, or any other unit, the number should be written without specifying the unit. For example, if you are asked for 100 millions, it should be written as 100, not 100 million or 100,000,000.
 - Never introduce a variable. For instance "gazelle_max_speed * 1.4" is not allowed. Pick up a correct number from the given context.

Here is an example of a correct answer:
If result1 is 100 and result2 is 50, you should output: |100 - 50| = 50.

 Question: {question}
 
"""

In [108]:
messages = [
    {"role":"system", "content":system_prompt},
    {"role":"user", "content":user_content}
]

response = ask(messages=messages,is_json=False)

print(response.choices[0].message.content)

|2794356 - 8258035| = 5463679


<h3>封装成类，方便调用<h3

In [109]:
class AbsDifference:
    def __init__(self, name:str="subtractions"):
        self.system_prompt = "You are a calculation assistant."

        self.name = name
        self.description = "math tool for calculating subtractions"
        
    def __call__(self, result1:str, result2:str) -> str:
        question = f"To calculate the abs difference between {result1} and {result2}"

        user_content = f"""
Answer the Question based on the Context. When you write down a expression, it MUST ONLY consists of numbers and operators. Here are some guidelines that you will be PANALIZED if you don't follow:

  - When you are asked for differences, you consider the absolute value of the difference. Difference of two numbers is always positive.For instance, the difference between 1 and 2 is 1, not -1.
  - When you are applying operations (e.g. difference, summation, ratio, etc.) between multiple values in the Context, you must unify the units of those numbers. For instance, you cannot add 1 meter to 1 foot.
     - You must pick the values in the same units if all the values are available in the same units.
     - If not, you must convert them to the same units before applying the operation.
  - You MUST strictly follow the unit (e.g. meter, kilometer, million, etc.) you were asked.
     - If the Context has the numbers in same units as the question, you can directly use them.
     - If the Context has the numbers in different units than the question, you must convert them to the units asked in the question.For example, if the question asks for the distance between two cities in kilometers, but the Context has the distance in miles, you must convert the distance to kilometers.
  - If you are asked about a particular number in millions, billions, or any other unit, the number should be written without specifying the unit. For example, if you are asked for 100 millions, it should be written as 100, not 100 million or 100,000,000.
 - Never introduce a variable. For instance "gazelle_max_speed * 1.4" is not allowed. Pick up a correct number from the given context.

Here is an example of a correct answer:
If result1 is 100 and result2 is 50, you should output: |100 - 50| = 50.

 Question: {question}
 
"""
        messages = [
            {"role":"system", "content":system_prompt},
            {"role":"user", "content":user_content}
        ]

        response = ask(messages=messages)
        return response.choices[0].message.content

In [110]:
math_tool = AbsDifference()
result = math_tool(result1, result2)
print(result)

|2794356 - 8258035| = 5463679
